In [7]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation,  MaxPooling2D, BatchNormalization
from keras import regularizers
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# # building the input vector from the 32x32 pixels
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# expand the input size by 2*2, so the input now is 64*64
X_train = np.repeat(X_train,2,axis=1)
X_train = np.repeat(X_train,2,axis=2)
X_test = np.repeat(X_test,2,axis=1)
X_test = np.repeat(X_test,2,axis=2)





# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()
L2_coeff = 0.0005
# convolutional layer
model.add(Conv2D(75, (3, 3), padding='same', activation='relu', input_shape=[64,64,3],kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(50, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(250, activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(Dense(250, activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
# output layer
model.add(Dense(10, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# model.fit(X_train, Y_train, batch_size=500, epochs=40, validation_data=(X_test, Y_test))
datagen = ImageDataGenerator(
            rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images
datagen.fit(X_train)
model.fit(datagen.flow(X_train, Y_train, batch_size=500), steps_per_epoch=100, epochs=50, validation_data=(X_test, Y_test))

Shape before one-hot encoding:  (50000, 1)
Shape after one-hot encoding:  (50000, 10)
Epoch 1/50
100/100 [==============================] - 109s 600ms/step - loss: 2.3327 - accuracy: 0.3586 - val_loss: 3.9457 - val_accuracy: 0.1000
Epoch 2/50
100/100 [==============================] - 59s 593ms/step - loss: 1.8018 - accuracy: 0.5021 - val_loss: 4.2492 - val_accuracy: 0.1000
Epoch 3/50
100/100 [==============================] - 59s 592ms/step - loss: 1.6300 - accuracy: 0.5505 - val_loss: 3.8784 - val_accuracy: 0.1805
Epoch 4/50
100/100 [==============================] - 60s 599ms/step - loss: 1.5161 - accuracy: 0.5819 - val_loss: 3.4425 - val_accuracy: 0.2052
Epoch 5/50
100/100 [==============================] - 60s 599ms/step - loss: 1.4361 - accuracy: 0.6050 - val_loss: 2.2007 - val_accuracy: 0.3422
Epoch 6/50
100/100 [==============================] - 59s 592ms/step - loss: 1.3893 - accuracy: 0.6184 - val_loss: 1.5639 - val_accuracy: 0.5441
Epoch 7/50
100/100 [=======================

In [11]:
model.fit(X_train, Y_train, batch_size=500, epochs=1, validation_data=(X_test, Y_test))
model.save('simple_cnn_expanded2.h5')

100/100 [==============================] - 28s 276ms/step - loss: 0.3632 - accuracy: 0.9834 - val_loss: 0.9853 - val_accuracy: 0.7900


In [12]:
model.summary( )

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 64, 64, 75)        2100      
_________________________________________________________________
batch_normalization_15 (Batc (None, 64, 64, 75)        300       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 64, 64, 50)        33800     
_________________________________________________________________
batch_normalization_16 (Batc (None, 64, 64, 50)        200       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 32, 32, 50)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 32, 32, 50)        22550     
_________________________________________________________________
batch_normalization_17 (Batc (None, 32, 32, 50)       